In [52]:
import numpy as np
import pandas as pd
import random as python_random
import shutil
import os
from os import listdir
import datetime
from pathlib import Path
from PIL import UnidentifiedImageError
from PIL import Image
from multiprocessing import cpu_count
from joblib import Parallel, delayed
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# for reproducibility purposes
SEED = 123
tf.random.set_seed(SEED)

# load tensorboard extension
%reload_ext tensorboard
# specify the log directory where the tensorboard logs will be written
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 25096), started 6 days, 22:44:23 ago. (Use '!kill 25096' to kill it.)

In [53]:
#commonly used batch size
batch_size = 50

# Used to augment images, the below augmentations are used to augment the images and output as numeric data for machine learning
train_img_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.1,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
    preprocessing_function=None,
    data_format=None,
    # splitting 20% of the training data into validation data
    validation_split=0.2,
    dtype=None
)
# gets all of the images and labels from the training folder and applies the above augmentations for the training subset of data
flowfromframeTrain = train_img_gen.flow_from_directory(
    directory='./input/train',
    target_size=(224, 224),
    color_mode='rgb',
    classes=None,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=SEED,
    save_to_dir=None,
    save_prefix='',
    save_format='',
    follow_links=False,
    subset='training',
    interpolation='nearest'
)
    
flowfromframeVal = train_img_gen.flow_from_directory(
    directory='./input/train',
    target_size=(224, 224),
    color_mode='rgb',
    classes=None,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=SEED,
    save_to_dir=None,
    save_prefix='',
    save_format='',
    follow_links=False,
    subset='validation',
    interpolation='nearest'
)

test_img_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=1./255,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None
)

flowfromframeTest = test_img_gen.flow_from_directory(
    directory='./input/test',
    target_size=(224, 224),
    color_mode='rgb',
    classes=None,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False,
    seed=SEED,
    save_to_dir=None,
    save_prefix='',
    save_format='',
    follow_links=False,
    interpolation='nearest'
)

Found 7462 images belonging to 16 classes.
Found 1859 images belonging to 16 classes.
Found 773 images belonging to 16 classes.


In [54]:
# VGG16 model is initialized here
# model = tf.keras.applications.vgg16.VGG16(
#     include_top=True,
#     weights=None,
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=22,
#     classifier_activation='softmax'
# )
input_shape=(224, 224, 3)
model = tf.keras.Sequential([
  
  layers.Conv2D(16, 3, padding='same', activation='relu',input_shape=input_shape),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(16, activation='softmax')
])

learn_rate = (0.001)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learn_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model_Callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3),
]

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='./data/model',
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
    options=None,
    initial_value_threshold=None,
)
model_Callbacks.append(model_checkpoint)

reduceLRCallback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=2,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0.0001,
)
model_Callbacks.append(reduceLRCallback)

numStepsPerEpoch = flowfromframeTrain.total_batches_seen
model.fit(
    x=flowfromframeTrain,
    y=None,
    batch_size=batch_size,
    epochs=20,
    verbose=1,
    callbacks=model_Callbacks,
    validation_split=0.0,
    validation_data=flowfromframeVal,
    shuffle=False,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=100,
    validation_steps=25,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)




Epoch 1/10
100/100 [==============================] - 85s 844ms/step - loss: 2.6308 - accuracy: 0.2174 - val_loss: 2.3123 - val_accuracy: 0.2432
INFO:tensorflow:Assets written to: ./data\model\assets
Epoch 2/10
100/100 [==============================] - 84s 837ms/step - loss: 2.1237 - accuracy: 0.3302 - val_loss: 1.9066 - val_accuracy: 0.3920
INFO:tensorflow:Assets written to: ./data\model\assets
Epoch 3/10
100/100 [==============================] - 86s 850ms/step - loss: 1.9043 - accuracy: 0.4052 - val_loss: 1.6291 - val_accuracy: 0.5112
INFO:tensorflow:Assets written to: ./data\model\assets
Epoch 4/10
100/100 [==============================] - 85s 840ms/step - loss: 1.7452 - accuracy: 0.4616 - val_loss: 1.6246 - val_accuracy: 0.4952
INFO:tensorflow:Assets written to: ./data\model\assets
Epoch 5/10
100/100 [==============================] - 85s 843ms/step - loss: 1.6443 - accuracy: 0.4922 - val_loss: 1.4555 - val_accuracy: 0.5280
INFO:tensorflow:Assets written to: ./data\model\assets


In [55]:
scores = model.evaluate(flowfromframeTest)


predictions = model.predict(flowfromframeTest)

test_labels = flowfromframeTest.labels

y_pred = np.argmax(predictions,axis=-1)

print(classification_report(test_labels, y_pred))

c_matrix = confusion_matrix(test_labels, y_pred)
print(c_matrix)

16/16 [==============================] - 4s 207ms/step - loss: 1.5714 - accuracy: 0.5175
              precision    recall  f1-score   support

           0       0.62      0.42      0.50        38
           1       0.36      0.50      0.42        38
           2       0.61      0.62      0.62        45
           3       0.63      0.51      0.56        43
           4       0.65      0.74      0.69        57
           5       0.49      0.74      0.59        46
           6       0.41      0.23      0.30        52
           7       0.48      0.51      0.49        41
           8       0.62      0.73      0.67        55
           9       0.38      0.28      0.32        40
          10       0.88      0.83      0.86        54
          11       0.46      0.48      0.47        56
          12       0.61      0.48      0.54        62
          13       0.38      0.44      0.41        54
          14       0.24      0.19      0.21        47
          15       0.38      0.44      0.41   